In [1]:
import graphlab as gl
import graphlab.aggregate as agg
gl.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)

nyt = gl.SFrame('nyt.sf')
aca = gl.SFrame('academic_articles.sf')

[INFO] This non-commercial license of GraphLab Create is assigned to loganwls@uw.edu and will expire on October 23, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-127287 - Server binary: /home/loganwls/.local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1463256954.log
[INFO] GraphLab Server Version: 1.6.1


In [2]:
# Make the node list

# For NYT we'll add author's most frequent topic
nyt_nodes = nyt.groupby(['author', 'topic_id'], {'count': agg.COUNT()}).groupby('author', {'most_frequent_topic': agg.ARGMAX('count', 'topic_id')})
nyt_nodes['average_eigenfactor'] = gl.SArray.from_const(None, nyt_nodes.num_rows()).astype(float)

# For academic we'll add the author's average eigenfactor
aca_nodes = aca[['author', 'eigenfactor']].stack('author', 'author').groupby('author', {'average_eigenfactor': agg.MEAN('eigenfactor')})
aca_nodes['most_frequent_topic'] = gl.SArray.from_const(None, aca_nodes.num_rows()).astype(int)

nodes = nyt_nodes.append(aca_nodes).add_row_number('id')
nodes

id,author,most_frequent_topic,average_eigenfactor
0,JULIE CRESWELL,2,None
1,STEVEN PETROW,1,None
2,"PERRI KLASS, M.D",1,None
3,JANE E. BRODY,1,None
4,DAVID WALLIS,1,None
5,LINDSAY CROUSE,3,None
6,DOUGLAS QUENQUA,3,None
7,JEN A. MILLER,3,None
8,ANDREW POLLACK,1,None
9,ASHLEY TAYLOR,3,None


In [3]:
nodes.save('author_network/nodes.csv')

In [4]:
# Create the edge list
author_to_author = nyt[['author', 'DOI']].stack('DOI', 'DOI').join(aca[['author', 'DOI']], on='DOI', how='inner').stack('author.1', 'aca_author')
del author_to_author['DOI']
edges = author_to_author.join(nodes['id', 'author'], on='author', how='left')
edges.rename({'id':'source'})
edges = edges.join(nodes['id', 'author'], on={'aca_author':'author'}, how='left')
edges.rename({'id':'target'})
edges.remove_columns(['author', 'aca_author'])
edges

source,target
72,5632
72,4509
72,4640
72,2469
72,4031
48,4819
48,6837
48,7660
48,6698
48,4819


In [5]:
edges.save('author_network/edges.csv')